### Retriver and Chain with Langchain

In [1]:
# we will be using previous embedding stored in weviate vector store

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_weaviate.vectorstores import WeaviateVectorStore
from weaviate.auth import Auth
import os
import weaviate

client = weaviate.connect_to_weaviate_cloud(
            cluster_url=f'https://{os.getenv("WEAVIATE_URL")}',
            auth_credentials=Auth.api_key(os.getenv('WEAVIATE_API_KEY')),
            headers={
                    'X-OpenAI-Api-Key': os.getenv('OPENAI_API_KEY') # openai api key for vectorizer & generative
                }
        )

open_ai_embeddings = OpenAIEmbeddings()

In [2]:
db_conection = WeaviateVectorStore(client=client, index_name="Attention", text_key="content", embedding=open_ai_embeddings)

In [3]:
from langchain_openai import OpenAI

# load the openAI model
openai_llm = OpenAI()

In [5]:
# desgin the prompt template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following qestions based only on the provided context.
think step by step before providing a detailed answer.
I will tip you $100 if the user find the answer helpful.
<context>
{context}
</context>
Question: {input}""")




In [6]:
## chain introduction

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(openai_llm, prompt)

In [ ]:
"""
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store.
A retriever does not need to be able to store documents, only to return (or retrieve) them. Retrievers can be created from vector stores,
but are also broad enough to include Wikipedia search and Amazon Kendra.

Retrievers accept a string query as input and return a list of Documents as output.
hybrid_search ={
    search_type="hybrid",
    search_kwargs={
        "alpha": 0.7,    # 70% vector, 30% keyword
        "k": 5,
        "query": "LangChain integration with RAG"
    }
"""

retriver = db_conection.as_retriever(
            search_type="mmr",  # or "similarity"
            search_kwargs={"k": 5, "lambda_mult": 0.7}
        )
retriver

VectorStoreRetriever(tags=['WeaviateVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_weaviate.vectorstores.WeaviateVectorStore object at 0x000002A14B89B2F0>, search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 0.7})

In [ ]:
from langchain.chains import create_retrieval_chain
retriver_chain = create_retrieval_chain(retriver, document_chain)
response = retriver_chain.invoke({'input': 'give me the function for attention mechanisim'})
response['answer']

' in the context of multi-head attention\nAnswer: The attention function for multi-head attention can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of the values, where the weights are determined by the compatibility between the query and each key. This compatibility can be computed using either additive attention or dot-product attention, with the latter being more efficient due to the use of a scaling factor of 1/√dk.'